In [1]:
import os
import requests
import json
import urllib3
from password_manager import PasswordManager

In [2]:
# deshabilitar la advertencia de InsecureRequestWarning
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
#Definicion de Variables
config_file = "config_encrypted.json"

In [4]:
# Funcion para leer configuracion desde JSON
def load_config(config_file):
    with open(config_file, "r") as file:
        return json.load(file)

In [5]:
def get_filtered_results(url, headers, params, fields, cert_file):
    all_filtered_results = []
    page = 1
    total_pages = None

    while total_pages is None or page <= total_pages:
        response_data = requests.get(f"{url}?page={page}", headers=headers, params=params, verify=False)
        response_data_json = response_data.json()

        if response_data.status_code != 200:
            print(f"Error: {response_data.status_code}")
            print(response_data.text)  # Detalles del error
            break

        if total_pages is None:
            total_pages = response_data_json['paging_info']['page_entries']
            print("TOTAL PAGES: ", total_pages)
        
        print(f"PAGE NUMBER: {page}")

        content_entries = response_data_json.get('alert_list', [])
        filtered_results = filter_entries(content_entries, fields)
        all_filtered_results.extend(filtered_results)

        page += 1  # Incrementar la pgina para la siguiente iteracin

    return all_filtered_results

In [6]:
def get_value_from_nested_keys(data, keys):
    for key in keys:
        if not isinstance(data, dict):
            return None
        data = data.get(key)
    return data

In [7]:
def filter_entries(entries, fields):
    filtered_results = []
    for entry in entries:
        filtered_entry = {}
        for field in fields:
            keys = field.split('.')
            value = get_value_from_nested_keys(entry, keys)
            filtered_entry[field] = value
        filtered_results.append(filtered_entry)
    return filtered_results

In [8]:
# Guardar los datos en un archivo JSON
def save_json(data, system, instance, query_name, base_path):
    output_file = os.path.join(base_path, f"{system}-{instance}-{query_name}")
    with open(output_file, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Datos guardados en: {output_file}")

In [9]:
#Funcion para obtener el token
def get_token_DD(instance, username, encrypted_password, cert_file):
    url = f'https://ddve-01:3009/rest/v1.0/auth'
    headers = {
        'Content-Type': 'application/json'
    }

    # Crear instancia de PasswordManager y desencriptar la contraseña
    password_manager = PasswordManager()
    password = password_manager.decrypt_password(encrypted_password)
    print(instance, username, password)

    data = {
        "username": username,
        "password": password
    }

    response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)
    #response = requests.post(url, headers=headers, data=json.dumps(data), verify=cert_file)
    
    if response.status_code == 201:
        response_json = response.json()
        access_token = response.headers.get('X-DD-AUTH-TOKEN')
        return access_token
    else:
        print(f"Error: {response.status_code}")
    return None

In [ ]:
def dd_get_alerts (instance, access_token, cert_file):
    url = f'https://{instance}:3009/rest/v2/dd-systems/0/alerts'
    headers = {
        "X-DD-AUTH-TOKEN": access_token
    }
    # filter_expression = (
    #     'status ne "cleared" '
    # )
    #filter_expression = {"status eq cleared"}
    filter_expression = "status = active"
    params = {
        'filter': filter_expression
    }
    fields = [
        "status",
        "severity",
        "class",
        "alert_id",
        "event_id"
        "name",
        "msg",
        "description",        
        "additional_info",
        "action"
    ]

    return get_filtered_results(url, headers, params, fields, cert_file)

In [11]:
def dd_get_services (instance, access_token, cert_file):
    print("FUNCION GET SERVICES", access_token)
    url = f'https://{instance}:3009/rest/v1.0/dd-systems/0/services'
    headers = {
        "X-DD-AUTH-TOKEN": access_token
    }
    params = {}
    fields = [
        "name",
        "status"
    ]
    
    response_data = requests.get(url, headers=headers, params=params, verify=False)
    response_data_json = response_data.json()
    # response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)

    # filtrar campos
    content_entries = response_data_json.get('services', [])
    print("content entries: ", content_entries)
    filtered_results = filter_entries(content_entries, fields)    
    print("FILTERED_RESULTS: ", filtered_results)

    return filtered_results

In [12]:
def main():
    config = load_config(config_file)
    base_path = config["basePath"]  # Obtener la ruta base desde el archivo de configuración
    json_relative_path = config["jsonPath"]
    jsonPath = os.path.join(base_path, json_relative_path)

    for system, system_data in config["systems"].items():
        json_files = system_data["files"]["json"]  # Obtener los nombres de los archivos JSON del sistema
        for instance_info in system_data["instances"]:
            instance = instance_info["hostname"]
            username = instance_info["username"]
            encrypted_password = instance_info["encrypted_password"]


            if system == "DD":
                print(f'{instance} {username} {encrypted_password}')
                cert_relative_path = config['systems']['PPDM']['files']['cert']
                cert_file = os.path.join(base_path, cert_relative_path)  

                access_token = get_token_DD(instance, username, encrypted_password, cert_file)

                if not access_token:
                    print(f"Error: no se pudo obtener el token para {instance}.")
                    continue

                print("Fetching active alerts...")
                data = dd_get_alerts(instance, access_token, cert_file)
                save_json(data, system, instance, json_files["activeAlerts"], jsonPath)                            

                print("Fetching state of services...")
                data = dd_get_services(instance, access_token, cert_file)
                save_json(data, system, instance, json_files["services"], jsonPath)

In [13]:
if __name__ == "__main__":
    main()

ddve-01 DCOapi gAAAAABnRgJvvNv2BQW2u8Wdqdhp2AanSQWDO7UJYtOv43EDjPhIHeaB6d90ysW4OMTBp9h-PbwocOMYnVJyagmvheXP3X5CAQ==
ddve-01 DCOapi Password123!
Fetching active alerts...
Error: 400
{"details": "**** \"status != cleared\" doesn't match pattern \"^(\\s*(severity|class|msg|object_id|status|alert_gen_epoch|node_id|event_id|is_active_only|updated_epoch)\\s*=\\s*(\\S*|(\\([^,]*,[^\\)]*\\))|(\"([^\"]*(\\\")*)*\"))\\s+[aA][nN][dD]\\s+)*\\s*(severity|class|msg|object_id|status|alert_gen_epoch|node_id|event_id|is_active_only|updated_epoch)\\s*=\\s*(\\S*|(\\([^,]*,[^\\)]*\\))|(\"([^\"]*(\\\")*)*\"))\\s*$\".", "code": 5446}
Datos guardados en: C:\github\dailycheck-PPDM\jsonFiles\DD-ddve-01-active_alerts.json
Fetching state of services...
FUNCION GET SERVICES 07d8250b37f494730ebf39988be89182e
content entries:  [{'name': 'ntp', 'status': 'DISABLED', 'link': [{'rel': 'self', 'href': '/rest/v1.0/dd-systems/0/services/ntp'}]}, {'name': 'snmp', 'status': 'ENABLED', 'link': [{'rel': 'self', 'href': '/res